# This Visualizes Predictions generated from save predictions
## adapted from Ensemble.py
#### Go to the main function down below, change to the correct csv file predictions, and use correct ensemble method. I reccomend the WBF one

In [3]:
import cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.patches as patches
import numpy as np
from PIL import Image

In [2]:
def imgDetect(filename, iou_thr, skip_box_thr, sigma, method):
    #return list of 
    
    boxes_list = [] #Get from some json or CSV
    scores_list = [] #Get from the json or CSV
    labels_list = [] #Get from the json or CSV
                     #IDK make it up
    
    retDic = {}     #Create dictionary that we will return
    
    
    with open(filename) as csvfile:
        readCSV = csv.reader(csvfile, delimiter = ',')    #read the csv
        next(readCSV)                       # Skip the header
        for row in readCSV:                 # for each row 
            imgID = row[0]
            
            #print(imgID)
            
            boxes_list = row[1]         #make boxes_list = first part of each line
            scores_list = row[2]        #make scores_list second part of line
            labels_list = row[3]        #make labels_list 3rd part of line

      # This converts all these strings to lists to be passed to the funtions below                    
            boxes_list = ast.literal_eval(boxes_list)
            scores_list = ast.literal_eval(scores_list)
            labels_list = ast.literal_eval(labels_list)
            
       # THIS FIRST CHECKS IF THERE ARE NO PREDICTIONS AT ALL, AND CONTINUES IF SO      
            empty = True
            for dex in range(len(scores_list)): #if everything is empty continue
                if not not scores_list[dex]:   #if theres something there
                    empty = False
                    
            if empty == True:   # IF THERE ARE NO PREDICTIONS AT ALL 
                continue        # CONTINUE TO NEXT LINE
                                 
            deleteDex = []
            for i in range(len(scores_list)):  #Loop to check EACH prediction empty
                if not scores_list[i]:
                    deleteDex.append(i)  #if there is no prediction = delete  
               
            boxes_list_ret  = []          #create new lists to add nonempty lists to
            scores_list_ret = []
            labels_list_ret = []
            
            for dex in range(len(scores_list)):
                if dex not in deleteDex:                        #only add non empty lists to this list
                    boxes_list_ret.append(boxes_list[dex])           
                    scores_list_ret.append(scores_list[dex])
                    labels_list_ret.append(labels_list[dex])
            
           
            boxes_list = boxes_list_ret       #reset boxes_list ..scres..labels to original variable
            scores_list = scores_list_ret
            labels_list = labels_list_ret
            
            
            if not (scores_list):    # check AGAIN after deletions 
                continue             #to see if there are no predictions at all
            
        
        # Adjust number of weights based on number of non empty predictions
        
            weights = []                        
            
            for i in range(len(scores_list)):   # Loop through
                weights.append(1)           # append a 1 to weights for len
            #weights = None
          
        #Creates a dictionary to add as value to key for each image
           
            valueDic = {}     # Create a dictionary that will b value of id key
            valueDic["boxes"] = ensemble(iou_thr, skip_box_thr, sigma, method, weights, boxes_list, scores_list, labels_list)[0]
            
            valueDic["scores"] = ensemble(iou_thr, skip_box_thr, sigma, method, weights, boxes_list, scores_list, labels_list)[1]
            valueDic["labels"] = ensemble(iou_thr, skip_box_thr, sigma, method, weights, boxes_list, scores_list, labels_list)[2]
            
            retDic[imgID] = valueDic    #make value dic the value of the id key
            #break
    
    return retDic
    

In [ ]:
# Feel free to adjust these values
def pred2dict(filename, method):
    iou_thr = 0.5
    skip_box_thr = .0001
    sigma = 0.1
    #method = "weighted_boxes_fusion"      # change this to get different types of ensembling
    #print(imgDetect(filename, iou_thr, skip_box_thr, sigma, method))
    return imgDetect(filename, iou_thr, skip_box_thr, sigma, method)

In [ ]:
# THIS FUNCTION WILL TURN THE RELATIVE COORDINATES TO THE MATPLOT FORM (X1,Y1, WDITH, HEIGHT)
def rel_to_matplotlib(bbox, img_path):
    img = cv2.imread(img_path)
    shape = img.shape    #gets a tuple (height, width)
    IMheight = shape[0]    #sets height
    IMwidth = shape[1]
        
    x0 = bbox[0]* IMwidth        #THIS GETS THE FIRST X COORDINATE
    y0 = bbox[1] * IMheight       #THIS GETS BOTTOM LEFT Y COORDINATE
    
    absX = IMwidth * bbox[2]
    absY = IMheight * bbox[3]
    

    width = absX - x0
    height = absY - y0
    
    return [x0,y0,width, height]


In [4]:
# Classes of amenities Airbnb mostly cares about
subset = ["Toilet",
          "Swimming_pool",
          "Bed",
          "Billiard_table",
          "Sink",
          "Fountain",
          "Oven",
          "Ceiling_fan",
          "Television",
          "Microwave_oven",
          "Gas_stove",
          "Refrigerator",
          "Kitchen_&_dining_room_table",
          "Washing_machine",
          "Bathtub",
          "Stairs",
          "Fireplace",
          "Pillow",
          "Mirror",
          "Shower",
          "Couch",
          "Countertop",
          "Coffeemaker",
          "Dishwasher",
          "Sofa_bed",
          "Tree_house",
          "Towel",
          "Porch",
          "Wine_rack",
          "Jacuzzi"]

subset.sort()
print(subset)

['Bathtub', 'Bed', 'Billiard_table', 'Ceiling_fan', 'Coffeemaker', 'Couch', 'Countertop', 'Dishwasher', 'Fireplace', 'Fountain', 'Gas_stove', 'Jacuzzi', 'Kitchen_&_dining_room_table', 'Microwave_oven', 'Mirror', 'Oven', 'Pillow', 'Porch', 'Refrigerator', 'Shower', 'Sink', 'Sofa_bed', 'Stairs', 'Swimming_pool', 'Television', 'Toilet', 'Towel', 'Tree_house', 'Washing_machine', 'Wine_rack']


In [ ]:
import random
# Running this cell will show the ensemble predictins for 5 random images from validation set
if __name__ == "__main__":    
    method = "weighted_boxes_fusion"
    ensemblePredictions = pred2dict("../BigDogModel/predictions_output.csv", method)   #change this to change the csv it reads from
    for key in random.sample(ensemblePredictions.keys(), 5):
        #print(ensemblePredictions[key]["scores"])
        path = "../BigDogModel/validation/" + key    # Make the path variable
        bboxes = ensemblePredictions[key]['boxes']   #Get the list of bounding boxes
        
        im = np.array(Image.open(path))              #start with the visualization thing
        fig, ax = plt.subplots(1, figsize=(8, 8))
        ax.imshow(im)
        
        dex = 0                                     #This is to get the correct score for each bounding box
        
    
        
        for i in range(len(bboxes)):
            bbox = bboxes[i]
            label = ensemblePredictions[key]["labels"][i]
            label = subset[int(label)]
            
            color = np.random.rand(3,)
            newbbox = rel_to_matplotlib(bbox, path)
            rect = patches.Rectangle((newbbox[0],newbbox[1]), newbbox[2], newbbox[3], linewidth=3,edgecolor=color,facecolor='none')
            ax.add_patch(rect)
            ax.annotate(label + " "+ str(ensemblePredictions[key]["scores"][dex]), color = color, xy = (newbbox[0],newbbox[1]))
            dex +=1
            
        plt.show
        
        